# Import Libraries

In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [3]:
DATA_DIR = 'C:\\Users\\Carrt\\OneDrive\\Documents\\ITCS 5152\\project-11\\data'
MODEL_DIR = 'C:\\Users\\Carrt\\OneDrive\\Documents\\ITCS 5152\\project-11\\Models\\Thomas'

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Import Data

In [65]:
# Global Constants
CLASSES = 7
BATCH_SIZE = 2
VALIDATION_PERCENT = .2


In [75]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Will add random distortions and discolorations to artificially multiply our limited dataset
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                    validation_split=VALIDATION_PERCENT,
                                    rotation_range=30,
                                    samplewise_center=True,
                                    height_shift_range=30,
                                    width_shift_range=30
                                   )

data_t = data_generator.flow_from_directory(DATA_DIR, target_size=(224, 224), batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True, subset='training')
data_v = data_generator.flow_from_directory(DATA_DIR, target_size=(224, 224), batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True, subset='validation') 

Found 1321 images belonging to 7 classes.
Found 327 images belonging to 7 classes.


# Create Model

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

model = Sequential([
        InputLayer(input_shape=(224, 224, 3), dtype='float32'),
        Conv2D(16, (4,4), activation='relu', padding='same'),
        Conv2D(16, (4,4), activation='relu', padding='same'),
        Conv2D(16, (4,4), activation='relu', padding='same'),
        BatchNormalization(),
        AveragePooling2D(pool_size = (2,2)),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        AveragePooling2D(pool_size = (2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPool2D(pool_size = (2,2)),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPool2D(pool_size = (2,2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.1),
        Dense(16, activation='relu'),
        Dense(CLASSES, activation='softmax')
    ])


In [56]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 224, 224, 16)      784       
                                                                 
 conv2d_45 (Conv2D)          (None, 224, 224, 16)      4112      
                                                                 
 conv2d_46 (Conv2D)          (None, 224, 224, 16)      4112      
                                                                 
 batch_normalization_18 (Bat  (None, 224, 224, 16)     64        
 chNormalization)                                                
                                                                 
 average_pooling2d_8 (Averag  (None, 112, 112, 16)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_47 (Conv2D)          (None, 112, 112, 32)     

# Train Model

In [71]:
# Tunables
BATCH_SIZE = 2
LEARNING_RATE = 0.001
DECAY = 1e-5
MOMENTUM = .3

LOSS_FUNCTION = 'categorical_crossentropy'
LOSS_METRICS = ['accuracy']

EPOCHS = 100
EARLY_STOP_PATIENCE = 15

In [72]:
from tensorflow.keras import optimizers

sgd = optimizers.SGD(lr=LEARNING_RATE, decay=DECAY, momentum=MOMENTUM, nesterov=True)
model.compile(optimizer=sgd, loss=LOSS_FUNCTION, metrics=LOSS_METRICS)

In [73]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = os.path.join(MODEL_DIR, 'BestModel.hdf5'), monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [76]:
fit_history = model.fit(data_t, steps_per_epoch=data_t.samples // BATCH_SIZE, epochs=EPOCHS, validation_data= data_v, validation_steps= data_v.samples // BATCH_SIZE, callbacks=[cb_checkpointer, cb_early_stopper])
model.load_weights(os.path.join(MODEL_DIR, 'BestModel.hdf5'))

Epoch 1/100
660/660 [==============================] - 12s 18ms/step - loss: 0.0385 - accuracy: 0.9886 - val_loss: 0.5505 - val_accuracy: 0.8804
Epoch 2/100
660/660 [==============================] - 11s 17ms/step - loss: 0.0579 - accuracy: 0.9773 - val_loss: 0.7409 - val_accuracy: 0.8374
Epoch 3/100
660/660 [==============================] - 11s 17ms/step - loss: 0.0400 - accuracy: 0.9871 - val_loss: 0.6153 - val_accuracy: 0.8528
Epoch 4/100
660/660 [==============================] - 11s 17ms/step - loss: 0.0304 - accuracy: 0.9886 - val_loss: 0.7534 - val_accuracy: 0.8221
Epoch 5/100
660/660 [==============================] - 11s 17ms/step - loss: 0.0459 - accuracy: 0.9856 - val_loss: 0.7422 - val_accuracy: 0.8129
Epoch 6/100
660/660 [==============================] - 11s 17ms/step - loss: 0.0351 - accuracy: 0.9879 - val_loss: 0.6756 - val_accuracy: 0.8282
Epoch 7/100
660/660 [==============================] - 11s 17ms/step - loss: 0.0571 - accuracy: 0.9856 - val_loss: 0.6436 - val_ac

KeyboardInterrupt: 

In [77]:
model.load_weights(os.path.join(MODEL_DIR, 'BestModel.hdf5'))
model.save('model')

INFO:tensorflow:Assets written to: model\assets
